# LlamaIndex - Starter Tutorial (Local Models)

:::{note}
This tutorial is based on LlamaIndexofficial docs tutorial ([https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/](https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/#starter-tutorial-local-models))
:::

## Download data and folder setup

On the **Docker host side**, run the following to set up the `jetson-containers`' `/data` directory.

```bash
cd jetson-containers
mkdir -p data/documents/L4T-README
cp /media/jetson/L4T-README/*.txt data/documents/L4T-README/
```

This in turn creates the mounted volume `/data/documents/L4T-README` inside the container.<br> 
Your directory structure should look like this:

```
└── ./data/documents/L4T-README
    ├── INDEX.txt
    ├── README-usb-dev-mode.txt
    ├── README-vnc.txt
    └── README-wifi.txt
```

You can check this with running a following bash command in the following cell.

In [1]:
!ls -Rl /data/documents/L4T-README

/data/documents/L4T-README:
total 24
-rw-rw-r-- 1 1000 1000  1104 May  6 22:42 INDEX.txt
-rw-rw-r-- 1 1000 1000 11126 May  6 22:42 README-usb-dev-mode.txt
-rw-rw-r-- 1 1000 1000  3590 May  6 22:42 README-vnc.txt
-rw-rw-r-- 1 1000 1000  1940 May  6 22:42 README-wifi.txt


## Download the Llama2 model

Check if you have Llama2 model downloaded.

In [2]:
!ollama list

NAME                    	ID          	SIZE  	MODIFIED     
mxbai-embed-large:latest	468836162de7	669 MB	10 hours ago	
llama3:70b              	be39eb53a197	39 GB 	25 hours ago	
llama3:latest           	a6990ed6be41	4.7 GB	25 hours ago	
nomic-embed-text:latest 	0a109f422b47	274 MB	32 hours ago	
llama2:latest           	78e26419b446	3.8 GB	11 days ago 	
mistral:latest          	61e88e884507	4.1 GB	2 weeks ago 	
llama2:70b              	e7f6c06ffef4	38 GB 	2 weeks ago 	
llama2:13b              	d475bf4c50bc	7.4 GB	2 weeks ago 	


If not, run the following cell to download the Llama2 model using `ollama` command.

In [ ]:
!ollama pull llama3

## Load data and build an index

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.ollama import Ollama

import logging
import sys
# import llama_index.core

# llama_index.core.set_global_handler("simple")
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
reader = SimpleDirectoryReader(input_dir="/data/documents/L4T-README/")

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 4
> [SimpleDirectoryReader] Total files added: 4


In [5]:
documents = reader.load_data()

DEBUG:fsspec.local:open file: /data/documents/L4T-README/INDEX.txt
open file: /data/documents/L4T-README/INDEX.txt
DEBUG:fsspec.local:open file: /data/documents/L4T-README/README-usb-dev-mode.txt
open file: /data/documents/L4T-README/README-usb-dev-mode.txt
DEBUG:fsspec.local:open file: /data/documents/L4T-README/README-vnc.txt
open file: /data/documents/L4T-README/README-vnc.txt
DEBUG:fsspec.local:open file: /data/documents/L4T-README/README-wifi.txt
open file: /data/documents/L4T-README/README-wifi.txt


In [6]:
len(documents)

4

In [7]:
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

In [8]:
# ollama
Settings.llm = Ollama(model="llama3", request_timeout=300.0)

In [9]:
# Enlarge the chunk size from the default 1024
Settings.chunk_size = 800
Settings.chunk_overlap = 50

In [10]:
index = VectorStoreIndex.from_documents(documents)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 1==============================================...
> Adding chunk: 1==============================================...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 2==============================================...
> Adding chunk: 2==============================================...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Ethernet on Linux
----------------------------...
> Adding chunk: Ethernet on Linux
----------------------------...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: You may log into the system and run shell comma...
> Adding chunk: You may log into the system and run shell comma...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 3==============================================...
> Adding chunk: 3==============================================...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 4=======================================

In [11]:
index.storage_context.persist()

DEBUG:fsspec.local:open file: /opt/llama-index/storage/docstore.json
open file: /opt/llama-index/storage/docstore.json
DEBUG:fsspec.local:open file: /opt/llama-index/storage/index_store.json
open file: /opt/llama-index/storage/index_store.json
DEBUG:fsspec.local:open file: /opt/llama-index/storage/graph_store.json
open file: /opt/llama-index/storage/graph_store.json
DEBUG:fsspec.local:open file: /opt/llama-index/storage/default__vector_store.json
open file: /opt/llama-index/storage/default__vector_store.json
DEBUG:fsspec.local:open file: /opt/llama-index/storage/image__vector_store.json
open file: /opt/llama-index/storage/image__vector_store.json


In [12]:
!du -ah ./storage

4.0K	./storage/index_store.json
132K	./storage/default__vector_store.json
4.0K	./storage/graph_store.json
4.0K	./storage/image__vector_store.json
32K	./storage/docstore.json
180K	./storage


## Query your data

In [13]:
query_engine = index.as_query_engine()

In [14]:
response = query_engine.query(
    "What IPv4 address Jetson device gets assigned when connected to a PC with a USB cable? \
    And what file to edit in order to change the IP address to be assigned to Jetson itself in USB device mode? \
    Plesae state which section you find the answer for each question."
)

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
http://localhost:11434 "POST /api/embeddings HTTP/1.1" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 736f1eb2-a778-4e79-880f-a814e11c5d81] [Similarity score:             0.694787] 2======================================================================
                        ...
> [Node 029afa15-bbd3-4423-b1bc-3b78ea33c911] [Similarity score:             0.595595] Ethernet on Linux
----------------------------------------------------------------------
Two US...
> Top 2 nodes:
> [Node 736f1eb2-a778-4e79-880f-a814e11c5d81] [Similarity score:             0.694787] 2======================================================================
                        ...
> [Node 029afa15-bbd3-4423-b1bc-3b78ea33c911] [Similarity score:             0.595595] 

In [15]:
from IPython.display import Markdown, display

display(Markdown(f"<b>{response}</b>"))

<b>What IPv4 address Jetson device gets assigned when connected to a PC with a USB cable?

According to the context information, under the "Ethernet" protocol, Linux for Tegra assigns a static IPv4 address of 192.168.55.1 to Jetson, and runs a DHCP server to automatically assign an IPv4 address of 192.168.55.100 to your host machine (Section: "Linux for Tegra implements two types of Ethernet devices...").

So, the answer is: 192.168.55.1

What file to edit in order to change the IP address to be assigned to Jetson itself in USB device mode?

According to the context information, you can edit the file "/opt/nvidia/l4t-usb-device-mode/nv-l4t-usb-device-mode-config.sh" on Jetson to change the IPv4 network parameters (Section: "Changing the IPv4 Address").

So, the answer is: /opt/nvidia/l4t-usb-device-mode/nv-l4t-usb-device-mode-config.sh</b>